In [4]:
import requests
import folium
import json
# GeoJSON 파일 다운로드
url = 'https://raw.githubusercontent.com/southkorea/seoul-maps/master/kostat/2013/json/seoul_municipalities_geo_simple.json'
response = requests.get(url)
geo_data = response.json()

In [15]:
# 방법1. json 파일로 지도 시각화 하기
# 서울 지도 생성
m = folium.Map(location=[37.5665, 126.9780], zoom_start=12)

# 구별로 색상을 칠하는 함수
def style_function(feature):
    colors = {
        '종로구': 'purple',
        '중구': 'blue',
        '용산구': 'green',
        '성동구': 'red',
        '광진구': 'orange',
        '동대문구': 'pink',
        '중랑구': 'yellow',
        '성북구': 'lightblue',
        '강북구': 'lightgreen',
        '도봉구': 'lightred',
        '노원구': 'lightorange',
        '은평구': 'lightpink',
        '서대문구': 'lightyellow',
        '마포구': 'purple',
        '양천구': 'blue',
        '강서구': 'green',
        '구로구': 'red',
        '금천구': 'orange',
        '영등포구': 'pink',
        '동작구': 'yellow',
        '관악구': 'lightblue',
        '서초구': 'lightgreen',
        '강남구': 'lightred',
        '송파구': 'lightorange',
        '강동구': 'lightpink'
    }
    return {
        'fillColor': colors.get(feature['properties']['name'], 'gray'),
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0.6
    }

# 구별로 색상을 칠한 폴리곤 추가
folium.GeoJson(
    geo_data,
    style_function=style_function,
    tooltip=folium.GeoJsonTooltip(fields=['name'], aliases=['구 이름'])
).add_to(m)

# 지도 저장
m.save('seoul_folium_color_json.html')

In [20]:
# 방법2. 데이터프레임 데이터가 있다면 .. 
import pandas as pd

# GeoJSON 데이터를 데이터프레임으로 변환
data = []
for feature in geo_data['features']:
    district_name = feature['properties']['name']
    coordinates = feature['geometry']['coordinates'][0]
    for coord in coordinates:
        data.append([district_name, coord[0], coord[1]])

# 데이터프레임 생성
df = pd.DataFrame(data, columns=['구 이름', '경도', '위도'])
df

,구 이름,경도,위도
0,강동구,127.115196,37.557533
1,강동구,127.118796,37.557222
2,강동구,127.121469,37.559860
3,강동구,127.124353,37.561442
4,강동구,127.135939,37.565648
...,...,...,...
1369,종로구,126.954270,37.622033
1370,종로구,126.958852,37.622576
1371,종로구,126.961642,37.625693
1372,종로구,126.971352,37.627434


In [21]:
# 서울 지도 생성
df_m = folium.Map(location=[37.5665, 126.9780], zoom_start=12)

In [22]:
# 각 구별로 다른 색상 지정
colors = {
    '종로구': 'purple',
    '중구': 'blue',
    '용산구': 'green',
    '성동구': 'red',
    '광진구': 'orange',
    '동대문구': 'pink',
    '중랑구': 'yellow',
    '성북구': 'lightblue',
    '강북구': 'lightgreen',
    '도봉구': 'lightred',
    '노원구': 'lightorange',
    '은평구': 'lightpink',
    '서대문구': 'lightyellow',
    '마포구': 'purple',
    '양천구': 'blue',
    '강서구': 'green',
    '구로구': 'red',
    '금천구': 'orange',
    '영등포구': 'pink',
    '동작구': 'yellow',
    '관악구': 'lightblue',
    '서초구': 'lightgreen',
    '강남구': 'lightred',
    '송파구': 'lightorange',
    '강동구': 'lightpink'
}

In [25]:
# 구별 폴리곤 추가
for district in df['구 이름'].unique():
    district_data = df[df['구 이름'] == district]
    coordinates = district_data[['위도', '경도']].values.tolist()

    # 경계선 폴리곤 만들기
    folium.Polygon(
        locations=coordinates,
        color='black',
        weight=1,
        fill=True,
        fill_color=colors.get(district, 'gray'),
        fill_opacity=0.6,
        popup=district
    ).add_to(df_m)
    
    # 중심 좌표 계산 (위도와 경도의 평균값 사용)
    center_lat = district_data['위도'].mean()
    center_lon = district_data['경도'].mean()

    # 마커 추가하기
    folium.Marker(
        location=[center_lat, center_lon],
        popup=district,
        icon=folium.Icon(color='blue', icon='bolt')
    ).add_to(df_m)


# 지도 저장
df_m.save('seoul_folium_color_dataframe.html')